代码路径：python/dgl

In [1]:
import dgl

构造一张图，因此我们先来分析dgl.DGLGraph，
直接从代码路径里未找到dgl.DGLGraph
u = np.concatenate([src, dst])
v = np.concatenate([dst, src])
##Construct a DGLGraph
dgl.DGLGraph((u, v))

In [ ]:
#用以下方式查看dgl，源码如下，可以看到在dgl模块的__init__.py中引入了常用库
??dgl

Type:        module
String form: <module 'dgl' from 'D:\\Program Files\\anaconda3\\lib\\site-packages\\dgl\\__init__.py'>
File:        d:\program files\anaconda3\lib\site-packages\dgl\__init__.py
Source:     
"""DGL root package."""
# Windows compatibility
# This initializes Winsock and performs cleanup at termination as required
import socket

# Need to ensure that the backend framework is imported before load dgl libs,
# otherwise weird cuda problem happens
from .backend import load_backend

from . import function
from . import nn
from . import contrib
from . import container
from . import random

from ._ffi.runtime_ctypes import TypeCode
from ._ffi.function import register_func, get_global_func, list_global_func_names, extract_ext_funcs
from ._ffi.base import DGLError, __version__

from .base import ALL, NTYPE, NID, ETYPE, EID
from .batched_graph import *
from .batched_heterograph import *
from .convert import *
from .graph import DGLGraph
from .heterograph import DGLHeteroGraph
from .nodeflow import *
from .traversal import *
from .transform import *
from .propagate import *
from .udf import NodeBatch, EdgeBatch

#在graph.py中可以看到
DGL graph accepts graph data of multiple formats:

    * NetworkX graph,
    * scipy matrix,
    * DGLGraph.
DGL接受3种图格式输入，NetworkX，DGLGraph可以理解，scipy matrix个人认为是数据+结构的方式，本质上没有太大区别；

If the input graph data is DGLGraph, the constructed DGLGraph only contains its graph index.
这个好理解，即图数据未重新构造，仅引用原图；

构造边，方式比较灵活
Add one edge at a time,

>>> G.add_edge(0, 1)

or multiple edges,

>>> G.add_edges([1, 2, 3], [3, 4, 5])  # three edges: 1->3, 2->4, 3->5

or multiple edges starting from the same node,

>>> G.add_edges(4, [7, 8, 9])  # three edges: 4->7, 4->8, 4->9

or multiple edges pointing to the same node,

>>> G.add_edges([2, 6, 8], 5)  # three edges: 2->5, 6->5, 8->5

.. note:: Here we use pytorch syntax for demo. The general idea applies
    to other frameworks with minor syntax change (e.g. replace
    ``torch.tensor`` with ``mxnet.ndarray``).

>>> import torch as th
>>> G.add_edges(th.tensor([3, 4, 5]), 1)  # three edges: 3->1, 4->1, 5->1

NOTE: Removing nodes and edges is not supported by DGLGraph.
DGLGraph不支持remove操作

**Features:**

Both nodes and edges can have feature data. Features are stored as
key/value pair. The key must be #hashable while the value must be tensor
type. Features are batched on the first dimension.

In [8]:
import numpy as np

def build_karate_club_graph():
    # All 78 edges are stored in two numpy arrays. One for source endpoints
    # while the other for destination endpoints.
    src = np.array([1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10,
        10, 11, 12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21,
        25, 25, 27, 27, 27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
    dst = np.array([0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4,
        5, 0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23,
        24, 2, 23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23,
        29, 30, 31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30,
        31, 32])
    # Edges are directional in DGL; Make them bi-directional.
    u = np.concatenate([src, dst])
    v = np.concatenate([dst, src])
    # Construct a DGLGraph
    return dgl.DGLGraph((u, v))

In [9]:
G = build_karate_club_graph()
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

We have 34 nodes.
We have 156 edges.


先构造一张图，再来解释图的属性

if isinstance(graph_data, DGLGraph):
    gidx = graph_data._graph
    if sort_csr:
        gidx.sort_csr()
else:
    if multigraph is not None:
        dgl_warning("multigraph will be deprecated." \
                    "DGL will treat all graphs as multigraph in the future.")
    gidx = graph_index.create_graph_index(graph_data, readonly)
    if sort_csr:
        gidx.sort_csr()
super(DGLGraph, self).__init__(gidx)

对以上代码进行解析

In [16]:
G._graph

可以看到是graph_index下面的类
class GraphIndex(ObjectBase):
    """Graph index object.

#gidx.sort_csr()
def sort_csr(self):
    """Sort the CSR matrix in the graph index.

    By default, when the CSR matrix is created, the edges may be stored
    in an arbitrary order. Sometimes, we want to sort them to accelerate
    some computation. For example, `has_edges_between` can be much faster
    on a giant adjacency matrix if the edges in the matrix is sorted.
    """
    _CAPI_DGLSortAdj(self)
可以看到，sort_csr用来对稀疏矩阵进行排序，为了加速计算；